In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import random

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [2]:
p = 380
model_name = os.path.join(os.path.dirname(os.getcwd()), 'model', 'final_model.h5')
model = tf.keras.models.load_model(model_name)

In [3]:
df_img_name = os.path.join(os.path.dirname(os.getcwd()), 'preprocess', f"{'quiz_preprocess'}.csv")
df_img = pd.read_csv(df_img_name)

In [4]:
df_quiz_name = os.path.join(os.path.dirname(os.getcwd()), 'raw', f"{'quiz'}.csv")
df_quiz = pd.read_csv(df_quiz_name)

In [5]:
def returnX(X):
    
    """
    train데이터와 동일하게 DatraFrame의 row(p, )를 이미지(1, p, p, 3)로 변경
    """
    
    X = X.replace("[", "").replace("]", "")
    X = np.fromstring(X, sep = " ")
    an_array = np.resize(X, (p, 1))
    t_array = np.resize(X,(1, p))
    final_array = (np.dot(an_array, t_array) +1) / 2
    final_array = final_array.astype(np.float16)
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((p,p, 3))
    data = np.expand_dims(data, axis=0)
    
    return data

In [6]:
Score = list()
from tqdm.auto import tqdm
for i in tqdm(range(len(df_quiz))):
    
    X = returnX(df_img[df_img['cst_id_di'] == df_quiz.cst_id_di[i]]["image"].to_string(index=False))
    y = df_quiz.loc[i, 'MRC_ID_DI']
    pred = model.predict(X)
    Score.append(pred[0][y])
    
df_quiz['Score'] = Score 

/home/cym_7172/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  


In [7]:
df_quiz.to_csv('quiz_s.csv')